In [1]:
import tensorflow as tf
import numpy as np
import gensim
import json
from gensim.models import Word2Vec

C:\Users\Dr. Anil Kumar Rai\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\Dr. Anil Kumar Rai\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from pprint import pprint

In [3]:
#read the data in list1,list2 and label lists
obj=None
list1 = []
seqlen1 = []
list2 = []
seqlen2 = []
label = []
with open("C:\\Users\\Dr. Anil Kumar Rai\\Desktop\\snli_1.0\\snli_1.0_train.jsonl","r") as f:
    for line in f:
        obj=json.loads(line)
        sent1=obj['sentence1'].split()
        sent2=obj['sentence2'].split()
        seqlen1.append(len(sent1))
        seqlen2.append(len(sent2))
        lab=obj['gold_label']
        list1.append(sent1)
        list2.append(sent2)
        label.append(lab)


In [4]:
#Train word2vec model
w2vlist=list1+list2
model = Word2Vec(w2vlist, size=200, window=5, min_count=1)

In [5]:
# create vocabulary for word2vec
vocab={}
ind=0
for line in w2vlist:
    for word in line:
        if word not in vocab:
            vocab[word]=ind
            ind=ind+1
print(len(vocab))

63037


In [6]:
vocab_size=len(vocab)
emb_size=200
embeddings = np.zeros((vocab_size, emb_size),dtype=np.float32)
for k, v in vocab.items():
  embeddings[v] = model[k]


C:\Users\Dr. Anil Kumar Rai\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [7]:
#put premises in text_train and hypothesis in hypo_train
text_train=[]
hypo_train=[]

train_lm=[]
test_lm=[]
for line in list1:
    new_lis=[]
    for word in line:
        new_lis.append(vocab[word])
        train_lm.append(vocab[word])
    text_train.append(new_lis)

for line in list2:
    new_lis=[]
    for word in line:
        new_lis.append(vocab[word])
        test_lm.append(vocab[word])
    hypo_train.append(new_lis)

In [8]:
train_lm[0:50]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 3,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 3,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 3,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 10,
 11,
 12,
 13,
 14,
 15,
 10,
 11,
 12,
 13,
 14]

In [9]:
#data path to store session
data_path="C\\Users\\Dr. Anil Kumar Rai\\Desktop\\lm"

In [10]:
#produce a batch of data
def batch_producer(raw_data, batch_size, num_steps):
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)
    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0: batch_size * batch_len],[batch_size, batch_len])
    epoch_size = (batch_len - 1) // num_steps
    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = data[:, i * num_steps:(i + 1) * num_steps]
    x.set_shape([batch_size, num_steps])
    y = data[:, i * num_steps + 1: (i + 1) * num_steps + 1]
    y.set_shape([batch_size, num_steps])
    return x, y


In [11]:
#to take input
class Input(object):
    def __init__(self, batch_size, num_steps, data):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
        self.input_data, self.targets = batch_producer(data, batch_size, num_steps)
        

In [12]:
import datetime as dt
#Creates bidirectional LSTM model
class Model(object):
    def assign_lr(self, session, lr_value):
            session.run(self.lr_update, feed_dict={self.new_lr: lr_value})

    def __init__(self, input, is_training, hidden_size, vocab_size, dropout=0.5, init_scale=0.05):
        self.is_training = is_training
        self.input_obj = input
        self.batch_size = input.batch_size
        self.num_steps = input.num_steps
        self.hidden_size = hidden_size

        with tf.device("/cpu:0"):
            inputs = tf.nn.embedding_lookup(embeddings, self.input_obj.input_data)
        
        if is_training and dropout < 1:
            inputs = tf.nn.dropout(inputs, dropout)
        self.init_state_fw = tf.placeholder(tf.float32, [2, self.batch_size, self.hidden_size])
        self.init_state_bw = tf.placeholder(tf.float32, [2, self.batch_size, self.hidden_size])
        n_init_state_fw = tf.unstack(self.init_state_fw, axis=0)
        n_init_state_bw = tf.unstack(self.init_state_bw, axis=0)
        rnn_tuple_state_fw = tf.contrib.rnn.LSTMStateTuple(n_init_state_fw[0],n_init_state_fw[1])
        rnn_tuple_state_bw = tf.contrib.rnn.LSTMStateTuple(n_init_state_bw[0],n_init_state_bw[1])
        cell_fw = tf.contrib.rnn.LSTMCell(self.hidden_size, state_is_tuple=True)
        cell_bw = tf.contrib.rnn.LSTMCell(self.hidden_size, state_is_tuple=True)
        cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=dropout)
        cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=dropout)
        ((encoder_outputs_fw,encoder_outputs_bw), (self.state_fw,self.state_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, inputs, initial_state_fw=rnn_tuple_state_fw, initial_state_bw=rnn_tuple_state_bw, dtype=tf.float32)
        encoder_outputs = tf.add(encoder_outputs_fw,encoder_outputs_bw) 
        output = tf.reshape(encoder_outputs, [-1, hidden_size])
        softmax_w = tf.Variable(tf.random_uniform([hidden_size, vocab_size], -init_scale, init_scale))
        softmax_b = tf.Variable(tf.random_uniform([vocab_size], -init_scale, init_scale))
        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        loss = tf.contrib.seq2seq.sequence_loss(
                    logits,
                    self.input_obj.targets,
                    tf.ones([self.batch_size, self.num_steps], dtype=tf.float32),
                    average_across_timesteps=False,
                    average_across_batch=True)
        self.cost = tf.reduce_sum(loss)
        self.softmax_out = tf.nn.softmax(tf.reshape(logits, [-1, vocab_size]))
        self.predict = tf.cast(tf.argmax(self.softmax_out, axis=1), tf.int32)
        correct_prediction = tf.equal(self.predict, tf.reshape(self.input_obj.targets, [-1]))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        if not is_training:
           return
        self.learning_rate = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), 5)
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        # optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.train_op = optimizer.apply_gradients(
            zip(grads, tvars),
            global_step=tf.contrib.framework.get_or_create_global_step())
        # self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(self.cost)

        self.new_lr = tf.placeholder(tf.float32, shape=[])
        self.lr_update = tf.assign(self.learning_rate, self.new_lr)


In [13]:
#function to train the BiLSTM Model
def train1(train_data, vocabulary, num_epochs, batch_size, model_save_name,
          learning_rate=1.0, max_lr_epoch=10, lr_decay=0.93, print_iter=50):
    # setup data and models
    training_input = Input(batch_size=batch_size, num_steps=35, data=train_data)
    m = Model(training_input, is_training=True, hidden_size=350, vocab_size=vocabulary)
    init_op = tf.global_variables_initializer()
    orig_decay = lr_decay
    with tf.Session() as sess:
        # start threads
        sess.run([init_op])
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        saver = tf.train.Saver()
        for epoch in range(num_epochs):
            new_lr_decay = orig_decay ** max(epoch + 1 - max_lr_epoch, 0.0)
            m.assign_lr(sess, learning_rate * new_lr_decay)
            current_state_fw = np.zeros((2, batch_size, m.hidden_size))
            current_state_bw = np.zeros((2, batch_size, m.hidden_size))
            curr_time = dt.datetime.now()
            for step in range(training_input.epoch_size):
                if step % print_iter != 0:
                    cost, _, current_state_fw, current_state_bw = sess.run([m.cost, m.train_op, m.state_fw, m.state_bw],
                                                      feed_dict={m.init_state_fw: current_state_fw, m.init_state_bw: current_state_bw})
                else:
                    seconds = (float((dt.datetime.now() - curr_time).seconds) / print_iter)
                    curr_time = dt.datetime.now()
                    cost, _, current_state_fw, current_state_bw, acc = sess.run([m.cost, m.train_op, m.state_fw, m.state_bw, m.accuracy],
                                                           feed_dict={m.init_state_fw: current_state_fw, m.init_state_bw: current_state_bw})
                    
                    print("Epoch {}, Step {}, cost: {:.3f}, accuracy: {:.3f}, Seconds per step: {:.3f}".format(epoch, step, cost, acc, seconds))

            # save a model checkpoint
            saver.save(sess, data_path + '\\' + model_save_name, global_step=epoch)
        # do a final save
        saver.save(sess, data_path + '\\' + model_save_name + '-final')
        # close threads
        coord.request_stop()
        coord.join(threads)

In [14]:
tf.reset_default_graph()
train1(train_lm,vocab_size,55,40,"model.ckpt")

UnboundLocalError: local variable 'encoder_outputs' referenced before assignment

In [ ]:
def test(model_path, test_data, reversed_dictionary):
    test_input = Input(batch_size=20, num_steps=35, data=test_data)
    m = Model(test_input, is_training=False, hidden_size=650, vocab_size=vocabulary,
              num_layers=2)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # start threads
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        current_state = np.zeros((2, 2, m.batch_size, m.hidden_size))
        # restore the trained model
        saver.restore(sess, model_path)
        # get an average accuracy over num_acc_batches
        num_acc_batches = 30
        check_batch_idx = 25
        acc_check_thresh = 5
        accuracy = 0
        for batch in range(num_acc_batches):
            if batch == check_batch_idx:
                true_vals, pred, current_state, acc = sess.run([m.input_obj.targets, m.predict, m.state, m.accuracy],
                                                               feed_dict={m.init_state: current_state})
                pred_string = [reversed_dictionary[x] for x in pred[:m.num_steps]]
                true_vals_string = [reversed_dictionary[x] for x in true_vals[0]]
                print("True values (1st line) vs predicted values (2nd line):")
                print(" ".join(true_vals_string))
                print(" ".join(pred_string))
            else:
                acc, current_state = sess.run([m.accuracy, m.state], feed_dict={m.init_state: current_state})
            if batch >= acc_check_thresh:
                accuracy += acc
        print("Average accuracy: {:.3f}".format(accuracy / (num_acc_batches-acc_check_thresh)))
        # close threads
        coord.request_stop()
        coord.join(threads)


if args.data_path:
    data_path = args.data_path
train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()
if args.run_opt == 1:
    train(train_data, vocabulary, num_layers=2, num_epochs=60, batch_size=20,
          model_save_name='two-layer-lstm-medium-config-60-epoch-0p93-lr-decay-10-max-lr')
else:
    trained_model = args.data_path + "\\two-layer-lstm-medium-config-60-epoch-0p93-lr-decay-10-max-lr-38"
    test(trained_model, test_data, reversed_dictionary)

INFO:tensorflow:Using C:\Users\DR8AF8~1.ANI\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with aggregation/weights
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' wit

INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias:0 from checkpoint b'C:\\Users\\DR8AF8~1.ANI\\AppData\\Local\\Temp\\tfhub_modules\\9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d\\variables\\variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell

TypeError: Can't convert 'tokens': Expected string, got 0 of type 'int' instead.